In [2]:
import warnings
warnings.filterwarnings("ignore")
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

2.9.1+cu126
True
NVIDIA GeForce RTX 5060 Ti


In [3]:
import torch.nn as nn 

In [37]:
class Model(nn.Module): 
    def __init__(self , num_features: int): 
        super().__init__()
        # Input layer
        self.linear1 = nn.Linear(in_features = num_features , out_features = 3)
        # add activation relu
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features = 3 , out_features = 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self , features): 
        # find the z = w*x + b 
        out = self.linear1(features)
        # pass the into ReLU
        out = self.relu(out)
        # pass to linear2 
        out = self.linear2(out) 
        out = self.sigmoid(out)
        return out

In [38]:
# create dataset witgh 5 featuers 
features = torch.rand(10 , 5 , dtype = torch.float32)
# make the model 
model = Model(features.shape[1])

# call the forward pass: pytorch recomand model(feaature) instead of model.forward(features)
model(features)

tensor([[0.4947],
        [0.4958],
        [0.4935],
        [0.4924],
        [0.4928],
        [0.4972],
        [0.4968],
        [0.4921],
        [0.4972],
        [0.4917]], grad_fn=<SigmoidBackward0>)

In [40]:
model.linear1.weight

Parameter containing:
tensor([[ 0.3507, -0.0783,  0.0321,  0.0988, -0.0357],
        [ 0.1120,  0.2748,  0.2818, -0.0493, -0.0226],
        [-0.2634, -0.3772,  0.3522, -0.3555,  0.2914]], requires_grad=True)

In [41]:
model.linear1.bias

Parameter containing:
tensor([-0.3916,  0.1764, -0.0913], requires_grad=True)

In [17]:
!pip install torchinfo

In [26]:
from torchinfo import summary

In [27]:
summary(model = model , input_size = (10 , 5))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [42]:
model

Model(
  (linear1): Linear(in_features=5, out_features=3, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## Better way to create NN using Sequential

In [44]:
class MyModel(nn.Module): 
    def __init__(self , num_features):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(num_features , 3), 
            nn.ReLU(), 
            nn.Linear(3 , 1), 
            nn.Sigmoid()
        )

    def forward(self , features): 
        out = self.network(features)
        return out 

In [45]:
model2 = MyModel(features.shape[1])
model2(features)

tensor([[0.5363],
        [0.5360],
        [0.5334],
        [0.5362],
        [0.5353],
        [0.5377],
        [0.5389],
        [0.5492],
        [0.5365],
        [0.5331]], grad_fn=<SigmoidBackward0>)

In [46]:
model2

MyModel(
  (network): Sequential(
    (0): Linear(in_features=5, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [49]:
df.drop(columns = ['id' , 'Unnamed: 32'] , axis = 1 , inplace = True)

In [50]:
# train test split 
X_train , X_test , y_train , y_test = train_test_split(df.iloc[ : , 1 : ] , df.iloc[ : , 0] , test_size = 0.2 , random_state = 42)

In [51]:
scaler = StandardScaler() 

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [53]:
# converting numpy to tensor 
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [59]:
X_train_tensor = X_train_tensor.float()
y_train_tensor = y_train_tensor.float()

In [74]:
X_test_tensor = X_test_tensor.float()
y_test_tensor = y_test_tensor.float()

In [64]:
# define the model 
class breastNN(nn.Module): 
    def __init__(self , num_features:int): 
        super().__init__()

        self.fc1 = nn.Linear(in_features = num_features , out_features = 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(in_features = 128 , out_features = 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features = 64 , out_features = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self , features): 
        out = self.fc1(features)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)

        return out

In [65]:
loss_function = nn.BCELoss()

In [70]:
epochs = 25
learning_rate = 0.1

breast_model = breastNN(X_train_tensor.shape[1])

for epoch in range(epochs): 
    # forward pass 
    y_pred = breast_model(X_train_tensor)
    # loss calculate 
    loss = loss_function(y_pred , y_train_tensor.view(-1 , 1))
    
    # backward pass 
    loss.backward()

    with torch.no_grad(): 
        breast_model.fc1.weight -= learning_rate * breast_model.fc1.weight.grad
        breast_model.fc1.bias -= learning_rate * breast_model.fc1.bias.grad

        breast_model.fc2.weight -= learning_rate * breast_model.fc2.weight.grad
        breast_model.fc2.bias -= learning_rate * breast_model.fc2.bias.grad

        breast_model.fc3.weight -= learning_rate * breast_model.fc3.weight.grad
        breast_model.fc3.bias -= learning_rate * breast_model.fc3.bias.grad
    
    # zero gradients
    breast_model.fc1.weight.grad.zero_()
    breast_model.fc2.weight.grad.zero_()
    breast_model.fc3.weight.grad.zero_()

    print(f"Epoch {epoch + 1}: Loss = {loss.item():.4f}")

Epoch 1: Loss = 0.6933
Epoch 2: Loss = 0.6703
Epoch 3: Loss = 0.6458
Epoch 4: Loss = 0.6202
Epoch 5: Loss = 0.5941
Epoch 6: Loss = 0.5680
Epoch 7: Loss = 0.5426
Epoch 8: Loss = 0.5185
Epoch 9: Loss = 0.4959
Epoch 10: Loss = 0.4746
Epoch 11: Loss = 0.4543
Epoch 12: Loss = 0.4343
Epoch 13: Loss = 0.4140
Epoch 14: Loss = 0.3931
Epoch 15: Loss = 0.3714
Epoch 16: Loss = 0.3492
Epoch 17: Loss = 0.3269
Epoch 18: Loss = 0.3051
Epoch 19: Loss = 0.2844
Epoch 20: Loss = 0.2652
Epoch 21: Loss = 0.2478
Epoch 22: Loss = 0.2323
Epoch 23: Loss = 0.2186
Epoch 24: Loss = 0.2067
Epoch 25: Loss = 0.1962


In [79]:
# model evaluation 
with torch.no_grad(): 
    y_pred = breast_model.forward(X_test_tensor)
    y_pred = (y_pred > 0.70).float()

    # print(y_pred.dtype)
    # print(y_test_tensor.dtype)
    
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy}")

Accuracy: 0.5517082214355469


In [80]:
breast_model.parameters()

<generator object Module.parameters at 0x00000199C2BFEEA0>

In [83]:
# using Optim from Pytorch 
import torch.optim as optim


epochs = 25
learning_rate = 0.1

breast_model1 = breastNN(X_train_tensor.shape[1])
# define the optimizer
optimizer = optim.SGD(params = breast_model1.parameters() , lr = learning_rate)

for epoch in range(epochs): 
    # forward pass 
    y_pred = breast_model(X_train_tensor)
    # loss calculate 
    loss = loss_function(y_pred , y_train_tensor.view(-1 , 1))

    # clear gradients
    optimizer.zero_grad()
    # backward pass 
    loss.backward()
    # update weights and bias 
    optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item():.4f}")

Epoch 1: Loss = 0.1870
Epoch 2: Loss = 0.1870
Epoch 3: Loss = 0.1870
Epoch 4: Loss = 0.1870
Epoch 5: Loss = 0.1870
Epoch 6: Loss = 0.1870
Epoch 7: Loss = 0.1870
Epoch 8: Loss = 0.1870
Epoch 9: Loss = 0.1870
Epoch 10: Loss = 0.1870
Epoch 11: Loss = 0.1870
Epoch 12: Loss = 0.1870
Epoch 13: Loss = 0.1870
Epoch 14: Loss = 0.1870
Epoch 15: Loss = 0.1870
Epoch 16: Loss = 0.1870
Epoch 17: Loss = 0.1870
Epoch 18: Loss = 0.1870
Epoch 19: Loss = 0.1870
Epoch 20: Loss = 0.1870
Epoch 21: Loss = 0.1870
Epoch 22: Loss = 0.1870
Epoch 23: Loss = 0.1870
Epoch 24: Loss = 0.1870
Epoch 25: Loss = 0.1870


In [84]:
# model evaluation 
with torch.no_grad(): 
    y_pred = breast_model1.forward(X_test_tensor)
    y_pred = (y_pred > 0.70).float()

    # print(y_pred.dtype)
    # print(y_test_tensor.dtype)
    
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy}")

Accuracy: 0.6228070259094238
